In [59]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

### This Script Gets the Billboard Year-End Songs

In [60]:
# Function to remove surrounding double quotes
def remove_double_quotes(s):
    if s.startswith('"') and s.endswith('"'):
        return s[1:-1]
    return s

def fetch_songs_for_year(year):
    url = f"https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    songs = []
    for i, song in enumerate(soup.findAll('tr')[1:101]): 
        song_artist_text = song.get_text()
        parts = song_artist_text.split('\n')
        if year<2021:
            songs.append((parts[1], remove_double_quotes(parts[2]), parts[3]))
        elif year>=2021:
            songs.append((parts[1], remove_double_quotes(parts[3]), parts[4]))
        if songs[i][2]=='':
            songs[i] = (songs[i][0], songs[i][1], songs[i-1][2])
    return songs


# Dictionary to hold all data
all_data = {}

# Loop through years
for year in range(1960, 2024):
    all_data[year] = fetch_songs_for_year(year)

# Writing data to CSV
with open('billboard_songs_1960_2023.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Position', 'Year', 'Song Title', 'Artist'])
    for year, songs in all_data.items():
        for position, title, artist in songs:
            writer.writerow([position, year, title, artist])

print("CSV file created successfully.")

CSV file created successfully.


In [61]:
billboard_df = pd.read_csv('billboard_songs_1960_2023.csv')

In [62]:
billboard_df

,Position,Year,Song Title,Artist
0,1,1960,Theme from A Summer Place,Percy Faith
1,2,1960,He'll Have to Go,Jim Reeves
2,3,1960,Cathy's Clown,The Everly Brothers
3,4,1960,Running Bear,Johnny Preston
4,5,1960,Teen Angel,Mark Dinning
...,...,...,...,...
6395,95,2023,I Wrote the Book,Morgan Wallen
6396,96,2023,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira
6397,97,2023,Meltdown,Travis Scott featuring Drake
6398,98,2023,Put It on da Floor Again,Latto featuring Cardi B


In [63]:
billboard_df.isna().sum()

Position      0
Year          0
Song Title    2
Artist        1
dtype: int64

In [64]:
billboard_df[billboard_df.isna().any(axis=1)]

,Position,Year,Song Title,Artist
5200,No.,2012,NaN,Title
5300,No.,2013,NaN,Title
6300,No.,2023,Title,NaN


In [65]:
added_songs = columns = {'Position':['100', '100', '100'], 'Year':[2012, 2013, 2023], 
                                    'Song Title':['Burn It Down', 'Still Into You', 'Watermelon Moonshine'], 
                                    'Artist':['Linkin Park','Paramore','Lainey Wilson']}
added_rows = pd.DataFrame(added_songs)
added_rows

,Position,Year,Song Title,Artist
0,100,2012,Burn It Down,Linkin Park
1,100,2013,Still Into You,Paramore
2,100,2023,Watermelon Moonshine,Lainey Wilson


In [66]:
billboard_df = pd.concat([billboard_df, added_rows], ignore_index = True)
billboard_df = billboard_df.dropna()
billboard_df.reset_index(drop = True, inplace = True)

In [67]:
billboard_df  = billboard_df[['Year', 'Position', 'Song Title', 'Artist']]
billboard_df['Year'] = billboard_df['Year'].astype(int)
billboard_df['Position'] =billboard_df['Position'].astype(int) 

In [68]:
billboard_df = billboard_df.sort_values(by = ['Year', 'Position'])

In [69]:
billboard_df

,Year,Position,Song Title,Artist
0,1960,1,Theme from A Summer Place,Percy Faith
1,1960,2,He'll Have to Go,Jim Reeves
2,1960,3,Cathy's Clown,The Everly Brothers
3,1960,4,Running Bear,Johnny Preston
4,1960,5,Teen Angel,Mark Dinning
...,...,...,...,...
6393,2023,96,"Bzrp Music Sessions, Vol. 53",Bizarrap and Shakira
6394,2023,97,Meltdown,Travis Scott featuring Drake
6395,2023,98,Put It on da Floor Again,Latto featuring Cardi B
6396,2023,99,Bloody Mary,Lady Gaga


In [70]:
billboard_df.to_csv('billboard_songs_1960_2023.csv', index = None)